# Task

We have a rectangular cake with some raisins on it:

cake = 
  ........
  ..o.....
  ...o....
  ........
// o is the raisins

We need to cut the cake evenly into n small rectangular pieces, so that each small cake has 1 raisin. n is not an argument, it is the number of raisins contained inside the cake:

cake = 
  ........
  ..o.....
  ...o....
  ........

result should be an array:
  [
     ........
     ..o.....
  ,
     ...o....
     ........
  ]
// In order to clearly show, we omit the quotes and "\n"

If there is no solution, return an empty array []

# Note

    The number of raisins is always more than 1 and less than 10.
    If there are multiple solutions, select the one with the largest width of the first element of the array. (See also the examples below.)
    Evenly cut into n pieces, meaning the same area. But their shapes can be different. (See also the examples below.)
    In the result array, the order of pieces is from top to bottom and from left to right (according to the location of the upper left corner).
    Each piece of cake should be rectangular.

# Examples

    An example of multiple solutions:

cake = 
  .o......
  ......o.
  ....o...
  ..o.....

In this test case, we can found three solution:
solution 1 (horizontal cutting):
  [
    .o......  //piece 1
  ,
    ......o.  //piece 2
  ,
    ....o...  //piece 3
  ,
    ..o.....  //piece 4
  ]

solution 2 (vertical cutting):
  [
    .o  //piece 1
    ..
    ..
    ..
  ,
    ..  //piece 2
    ..
    ..
    o.
  ,
    ..  //piece 3
    ..
    o.
    ..
  ,
    ..  //piece 4
    o.
    ..
    ..
  ]

solution 3 (cross cutting):
  [
    .o..  //piece 1
    ....
  ,
    ....  //piece 2
    ..o.
  ,
    ....  //piece 3
    ..o.
  ,
    o...  //piece 4
    ....   
  ]

we need choose solution 1 as result

    An example of different shapes:

cake = 
  .o.o....
  ........
  ....o...
  ........
  .....o..
  ........

the result should be:
  [
    .o      //pieces 1
    ..
    ..
    ..
    ..
    ..
  ,
    .o....  //pieces 2
    ......
  ,
    ..o...  //pieces 3
    ......
  ,
    ...o..  //pieces 4
    ......   
  ]
Although they have different shapes, 
they have the same area(2 x 6 = 12 and 6 x 2 = 12).

    An example of no solution case:

cake = 
  .o.o....
  .o.o....
  ........
  ........
  ........
  ........
the result should be []

In [222]:
def get_slice( cake_rows, size_of_slice, width_of_slice ):
    # calculate depth of slice
    depth_of_slice = int(size_of_slice / width_of_slice)

    # look for first row in cake with unused positions
    sliced_cake = []
    for start_row in range(len(cake_rows)):
        spaces_left = len(cake_rows[start_row]) - cake_rows[start_row].count('x')
        if spaces_left > 0:
            # if it doesn't have enough unused positions to fit a slice of this width, return
            if spaces_left < width_of_slice:
                return [], cake_rows, 0
            # otherwise check the spaces are consecutive
            else:
                for start_col in range(len(cake_rows[start_row])):
                    if cake_rows[start_row][start_col] != 'x':
                        break
            break
        # initialise the sliced cake with used rows
        sliced_cake.append(cake_rows[start_row])

    # check there's sufficient depth to fit the slice
    if start_row + depth_of_slice > len(cake_rows):
        return [], cake_rows, 0

    # now extract the slice and replace slice entries with x in the cake to mark them as used
    this_slice = []
    for i in range(depth_of_slice):
        cake_row = cake_rows[start_row + i]
        slice_row = ''
        adj_cake_row = cake_row[:start_col]
        for j in range(start_col, start_col + width_of_slice):
            if cake_row[j] == 'x':
                return [], cake_rows, 0
            else:
                slice_row += cake_row[j]
                adj_cake_row += 'x'
        this_slice.append(slice_row)
        adj_cake_row += cake_row[start_col + width_of_slice :]
        sliced_cake.append(adj_cake_row)

    # check the slice only contains one raisin
    if ''.join(this_slice).count('o') != 1:
        return [], cake_rows, 0

    # append remaining rows to sliced cake
    for i in range(start_row + depth_of_slice, len(cake_rows)):
        sliced_cake.append( cake_rows[i] )

    # now again find the first row with unused positions
    for row in range(len(sliced_cake)):
        if len(sliced_cake[row]) - sliced_cake[row].count('x') > 0:
            break
    entry = sliced_cake[row]
    remaining_width = len(entry) - entry.count('x')

    return this_slice, sliced_cake, remaining_width

In [223]:
def cut(cake):
    # split cake into rows
    cake_rows = cake.split(chr(10))

    # dimensions of cake
    width_of_cake = len(cake_rows[0])
    height_of_cake = len(cake_rows)
    size_of_cake = width_of_cake * height_of_cake 
    n = cake.count('o')
    size_of_slice = size_of_cake / n

    # if size of slice is not an integer, there is no solution
    if size_of_slice != int(size_of_slice):
        return []
    size_of_slice = int(size_of_slice) 

    # # work through all possible slice widths, starting with the longest
    start_widths = []
    result = []
    sliced_cake = cake_rows[:]
    remaining_width = width_of_cake
    remaining_raisins = ''.join(sliced_cake).count('o')
    while remaining_raisins > 0 and remaining_width > 0:
        possible_widths = [i for i in range(remaining_width, 0, -1) if size_of_slice % i == 0]
        for width in possible_widths:
            # if we failed on a previous attempt, and this is the final iteration on which we got a valid slice last time, omit that slice width
            if len(start_widths) > 0:
                if len(result) < len(start_widths) - 1 and width > start_widths[len(result)]:
                    continue
                if len(result) == len(start_widths) - 1 and start_widths[len(result)] <= width:
                    continue
            # otherwise continue as normal
            if ''.join(sliced_cake).count('o') > 0:
                this_slice, sliced_cake, remaining_width = get_slice(sliced_cake, size_of_slice, width)
                if len(this_slice) > 0:
                    result.append(chr(10).join(this_slice).strip())
                    remaining_raisins = ''.join(sliced_cake).count('o')
                    break

        # if successfully sliced, break out of the loop
        if size_of_cake == ''.join(sliced_cake).count('x'):
            break

        if width == 1 and (size_of_cake - ''.join(sliced_cake).count('x')) > 0:
            # if we didn't manage to get any results at all there's no solution
            if len(result) == 0 and len(start_widths) == 0:
                return []

            # otherwise have another go with different start lengths
            prev_start_widths = start_widths[:]
            start_widths = []
            for entry in result:
                if entry.find(chr(10)) == -1:
                    start_widths.append(len(entry))
                else:
                    start_widths.append(entry.find(chr(10)))
            if len(start_widths) == 0:
                 start_widths = prev_start_widths[:-1]
                 if len(start_widths) == 0:
                     return []
            sliced_cake = cake_rows[:]
            remaining_raisins = ''.join(sliced_cake).count('o')
            remaining_width = width_of_cake
            result = []
        
    # if we've tried all widths and there is still unused cake, there's no solution
    if width == 1 and (size_of_cake - ''.join(sliced_cake).count('x')) > 0:
        return []

    return result

In [224]:
cake = '''
...............
............o..
.o.............
...............
o..............
..........o....
...............
...o...........
'''.strip()
print(cut(cake))

# expected 
# [ ........ / ........ / .o...... / ,
# .... / .... / .... / .... / .... / ..o. ,
# ... / o.. / ... / ... / ... / ... / ... / ... ,
# ........ / o....... / ........ / ,
# ............ / ...o........ ]

['........\n........\n.o......', '....\n....\n....\n....\n....\n..o.', '...\no..\n...\n...\n...\n...\n...\n...', '........\no.......\n........', '............\n...o........']


In [225]:
cake = '''
...................................................o..................
.....................o................................................
..........o................................o..........................
......................................................................
.........o.....o................................o.....................
'''.strip()
print(cut(cake))

# expected []

[]


In [226]:
cake = '''
.................................................................................
...........................o..............o.............................o........
........o........................................................................
.........o.......................................................................
...........o..............................o......................................
.................................o...................o........................... 
'''.strip()
print(cut(cake))

# expected []

[]


In [227]:
cake = '''
.........................................o......
................................................
..........o....o................................
................................o...............
............o...................................
.................................o..............
'''.strip()
print(cut(cake))

# EXPECTING
# [ ............ / ............ / ..........o. / ............ ,
# ............ / ............ / ...o........ / ............ ,
# .................o...... / ........................ ,
# ........................ / ........o............... ,
# ............o................................... ,
# .................................o.............. ]

['............\n............\n..........o.\n............', '............\n............\n...o........\n............', '.................o......\n........................', '........................\n........o...............', '............o...................................', '.................................o..............']


In [228]:
cake = '''
.o.o....
........
....o...
........
.....o..
........
'''.strip()
print(cut(cake))
# Expected:
# [ .o / .. / .. / .. / .. / .. ,
# .o.... / ...... ,
# ..o... / ...... ,
# ...o.. / ...... ]

['.o\n..\n..\n..\n..\n..', '.o....\n......', '..o...\n......', '...o..\n......']


In [229]:
cake = '''
..........o...................
..............................
..............................
..............................
..o..................o........
..............................
................o.............
..........o...................
.............................. 
'''.strip()
result = cut(cake)
print(result)
# expected []

[]


In [230]:
cake = '''
..........o.......................................
o................................................o
.o.................................o.............. 
'''.strip()
result = cut(cake)
print(result)

['..........o...................', '..........\n..........\n.....o....', '..........\n.........o\n..........', 'o.............................', '.o............................']


In [231]:
cake = '''
.o....o.
.o....o.
........
o..oo..o 
'''.strip()
print(cut(cake))
# SHOULD BE WIDTH 4, 4, 4, 4, 2, 2, 2, 2

['.o..', '..o.', '.o..', '..o.', '..\no.', '..\n.o', '..\no.', '..\n.o']


In [232]:
cake = '''
................
.....o..........
................
...............o
................
................
................
.....o..o.....o.
................
................
...o............
................
................
...............o
................
.o..............
'''.strip()
print(cut(cake))

['................\n.....o..........', '................\n...............o', '........\n........\n........\n.....o..', '....\n....\n....\no...\n....\n....\n....\n....', '....\n....\n....\n..o.\n....\n....\n....\n....', '........\n........\n...o....\n........', '................\n...............o', '................\n.o..............']


In [233]:
cake4 = '''
.o.o....
.o.o....
........
........
'''.strip()
print(cut(cake))

['................\n.....o..........', '................\n...............o', '........\n........\n........\n.....o..', '....\n....\n....\no...\n....\n....\n....\n....', '....\n....\n....\n..o.\n....\n....\n....\n....', '........\n........\n...o....\n........', '................\n...............o', '................\n.o..............']


In [234]:
cake = '''
.o....o.
.o....o.
........
o..oo..o
'''.strip()
print(cut(cake))

['.o..', '..o.', '.o..', '..o.', '..\no.', '..\n.o', '..\no.', '..\n.o']


In [235]:
cake = '''
........
..o.....
...o....
........
'''.strip()
print(cut(cake))

['........\n..o.....', '...o....\n........']
